In [1]:
import os, sys, time, psutil, subprocess
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pandas as pd
import json

In [ ]:
# set locations for working files
if len(sys.argv) != 5:
    print("Usage: python3 multi-sim.py <automation_dir> <simulators>")
    print('Assuming testing directories')
    automation_dir = "/mnt/analysis/e17023/Adam/GADGET2/"
    num_sims = 4
    premade = 0
else:
    # Automation directory
    automation_dir = sys.argv[1]
    
    # Number of simulators
    num_sims = sys.argv[2]
    
    tuning_mode = sys.argv[3]
    
    premade = sys.argv[4]
    
    try:
        num_sims = int(num_sims)
    except ValueError:
        print("Error: <simulators> must be an integer")
        sys.exit(1)
    if num_sims < 1:
        print("Error: <simulators> must be positive")
        sys.exit(1)
    elif num_sims > 10:
        print("Error: <simulators> must be less than 10")
        sys.exit(1)
        # highest tested value so far, deminishing returns?
        # probably should talk to IT before increasing this
    
    if tuning_mode == 'y':
        tuning_mode = True
    else:
        tuning_mode = False

In [ ]:
def start_sim(sim_dir, main_dir):
    if not os.path.isdir(sim_dir):
        os.system("mkdir " + sim_dir)
        #print(f"{time.strftime('%H:%M:%S')} | Creating directory {sim_dir}")
    else:
        # delete old output files
        os.system("rm -rf " + sim_dir + "out/*")
        os.system("rm -rf " + sim_dir + "*.tmp") # delete temporary files
        os.system("rm -rf " + sim_dir + "*.log") # delete log files

    
    # copy template files
    os.system("cp " + main_dir + "/.input/templates/* " + sim_dir)
    
    # convert queue and process notebooks to scripts
    os.system("python3 " + main_dir + "/.input/nb2py.py " + main_dir + ".input/sim/queue.ipynb")
    os.system("mv " + main_dir + ".input/sim/queue.py " + sim_dir)
    os.system("python3 " + main_dir + "/.input/nb2py.py " + main_dir + ".input/sim/process.ipynb")
    os.system("mv " + main_dir + ".input/sim/process.py " + sim_dir)
    os.system("python3 " + main_dir + "/.input/nb2py.py " + main_dir + ".input/sim/augment.ipynb")
    os.system("mv " + main_dir + ".input/sim/augment.py " + sim_dir)
    os.system("cp " + main_dir + ".input/sim/simGADGET.sh " + sim_dir)
    os.system("cp " + main_dir + ".input/sim/param.csv " + sim_dir)
    
    # start simulation in background and return
    os.chdir(sim_dir)
    os.system("chmod +x simGADGET.sh")
    # start simulation, nohup to keep running after logout and & to run in background, supress output
    os.system(f"nohup ./simGADGET.sh > {sim_dir}log.log 2>&1 &")
    os.chdir(main_dir)
    return None

In [ ]:
def display_status(prev_lines, statuses, automation_dir, tuning_mode, main_message=''):
    # clear previous lines
    for i in range(len(prev_lines)):
        print("\033[F\033[K", end='')
    
    
    lines = []
    lines.append(f"{'='*60}")
    lines.append(f"GADGET2 ATTPCROOT Parameters Automation")
    lines.append(f"{time.strftime('%H:%M:%S'):>20}")
    if main_message != "":
        lines.append(f"{'='*60}")
        lines.append(f"{main_message}")
    lines.append(f"{'='*60}")
    for i in range(len(statuses)):
        sim_name = ""
        try:
            sparam = pd.read_csv(f"{automation_dir}.sims/{i}/param.csv")
            if len(sparam) > 0:
                sim_name = sparam['Sim'][0]
            else:
                sim_name = ""
        except FileNotFoundError:
            sim_name = ""
        except pd.errors.EmptyDataError:
            sim_name = ""
        lines.append(f"Simulator {i}: {str(statuses[i].split('.')[0]): <20} | {sim_name}")
    lines.append(f"{'='*60}") 
    
    param_df = pd.read_csv(f"{automation_dir}parameters.csv")
    if tuning_mode == True:
        with open(automation_dir + 'tuning_log.json', 'r') as f:
            tuning_log = json.load(f)
        tot_sims = tuning_log['MaxIterations']
        for ptype in tuning_log['TuningParticles']:
            ptype_df = param_df[param_df['Sim'].astype(str).str.startswith(f'T{ptype}')]
            if len(ptype_df[ptype_df['Score'] != -1]) > 0:
                best_score = ptype_df[ptype_df['Score'] != -1]['Score'].min()
                best_sim = ptype_df[ptype_df['Score'] == best_score]['Sim'].values[0]
                lines.append(f"Best {ptype} Tuning Result: {best_score} from {best_sim}")
    else:
        tot_sims = len(param_df)
    lines.append(f"Completed {len(param_df[param_df['Status'] >= 2])} of {tot_sims} Simulations")
    
    print("\n".join(lines))
    return lines

In [ ]:
try:
    for simi in range(num_sims): # start all simulations in parallel
        start_sim(f'{automation_dir}.sims/{simi}/', automation_dir)
        #print(f"{time.strftime('%H:%M:%S')} | sim{simi} started")
    time.sleep(2) # give time for simulations to start

    iteration = 0
    while True: # simulation management loop
        if iteration == 0:
            lines = []
        if tuning_mode:
            # run tuning script, raise keyboard interupt if tuning script fails
            try:
                subprocess.run(["python3", f"{automation_dir}.input/tuning.py", automation_dir, str(num_sims), str(iteration), str(premade)], check=True)
            except subprocess.CalledProcessError:
                raise KeyboardInterrupt
           
        
        iteration = 1 # only iteration 0 once to get initial parameters
        
        statuses = [] # list of statuses for each simulation
        for simi in range(num_sims):        
            param_df = pd.read_csv(automation_dir + "parameters.csv") # read any updates to parameters.csv
            
            sim_dir = f'{automation_dir}.sims/{simi}/' # directory for current simulation
            try:
                sparam = pd.read_csv(sim_dir + "param.csv") # read individual parameter file for current simulation
            except pd.errors.EmptyDataError:
                # copy blank parameter file from main directory
                os.system("cp " + automation_dir + ".input/sim/param.csv " + sim_dir + "param.csv")
                sparam = pd.read_csv(sim_dir + "param.csv")
            
            # check each simulation for status files
            tmp_files = [0]
            for file in os.listdir(sim_dir):
                if file.endswith(".tmp"):
                    tmp_files.append(file)
            if tmp_files[-1] == 0:
                tmp_files = ['STOPPED.tmp']
            statuses.append(tmp_files[-1])
            
            # improperly stopped simulation, restart
            if statuses[-1] == 'STOPPED.tmp' and len(param_df[param_df['Status'] == 0]) >= num_sims:
                start_sim(f'{automation_dir}.sims/{simi}/', automation_dir)
            
            # if simulation is waiting, process output files and queue next run
            if statuses[-1] == 'WAIT.tmp':
                completed_sims = sparam[sparam['Status'] >= 2] # get list of completed sims
                for i in range(len(completed_sims)): # process each completed simulation
                    sim_name = completed_sims.iloc[i]['Sim']
                    # move sim's output files to main output directory
                    if os.path.isfile(f"{sim_dir}out/hdf5/{sim_name}.h5"):
                        os.system(f"mv -f {sim_dir}out/hdf5/{sim_name}.h5 {automation_dir}Output/hdf5/")
                    if len(os.listdir(f"{sim_dir}out/images/")) > 0:
                        os.system(f"mv -f {sim_dir}out/images/{sim_name}_* {automation_dir}Output/images/")
                    if len(os.listdir(f"{sim_dir}out/gifs/")) > 0:
                        os.system(f"mv -f {sim_dir}out/gifs/{sim_name}.gif {automation_dir}Output/gifs/")
                    
                    #print(f"{time.strftime('%H:%M:%S')} | sim{simi} completed {sim_name} with status {completed_sims.iloc[i]['Status']}")
                    param_df.loc[param_df['Sim'] == sim_name, 'Status'] = sparam.loc[sparam['Sim'] == sim_name, 'Status'].values[0]
                    param_df.to_csv(automation_dir + "parameters.csv", index = False)
                
                # queue next run if there are more sims to run
                if len(param_df[param_df['Status'] == 0]) > 0:
                    next_params = param_df[param_df['Status'] == 0].head(1) # get parameters for next sim
                    next_params.to_csv(sim_dir + "param.csv", index = False) # write parameters to sim's param.csv
                    param_df.loc[param_df['Sim'] == next_params['Sim'].iloc[0], 'Status'] = 1 # update status in parameters.csv
                    #print(f"{time.strftime('%H:%M:%S')} | sim{simi} running {next_params['Sim'].iloc[0]}") # print queued sim number
                else: # no more sims to queue, stop simulation iteration
                    with open(sim_dir + "STOP.tmp", "w") as f:
                        f.write(f"{time.strftime('%H:%M:%S')} stopped from manager")
                    #print(f"{time.strftime('%H:%M:%S')} | sim{simi} stopped")
            param_df.to_csv(automation_dir + "parameters.csv", index = False) # update parameters.csv with any changes

        status_df = pd.DataFrame({'Sim' : [simi for simi in range(num_sims)], 'Status' : [i.split('.')[0] for i in statuses]})
            
        lines = display_status(lines, statuses, automation_dir, tuning_mode)
        status_df.to_csv(automation_dir + "status.csv", index = False)
        
        if all([status == 'STOPPED.tmp' for status in statuses]):
            break
        time.sleep(1)
        
except KeyboardInterrupt:
    os.system(f"touch {automation_dir}.sims/STOP.tmp") # MASTER STOP FILE
    try:
        while True: # wait for all sims to stop
            statuses = []
            for simi in range(num_sims):
                tmp_files = [0]
                for file in os.listdir(sim_dir):
                    if file.endswith(".tmp"):
                        tmp_files.append(file)
                if tmp_files[-1] == 0:
                    tmp_files = ['STOPPED.tmp']
                statuses.append(tmp_files[-1])
                
            lines = display_status(lines, statuses, automation_dir, tuning_mode, 'Keyboard Interupt, Waiting for sims to stop') # continue to display status of remaining sims
            
            if all([status == 'STOPPED.tmp' for status in statuses]):
                sys.exit(0)
            
            time.sleep(1)
    except KeyboardInterrupt: # if keyboard interupt again, force stop
        print("Second Keyboard Interupt, forcing stop")
        # find all processes with name simGADGET.sh
        for proc in psutil.process_iter():
            if proc.name() == 'simGADGET.sh':
                proc.kill()
        print("All simulations force stopped")
        print("It is recommended reset sim directories with the -a flag for the next simulations")